In [14]:
# Implementation of 1-out-of-2 OT
# Alice has 2 messages and Bob wants to know one of those messages with Alice now knowing which message Bob learns and Bob not knowing the second of the two messages

import time
seed = int(round(time.time() * 10))

# Generator for random number generation
def gen_random_mod(k, modulus):
    num = 0
    while(num < k):
        a = mod(gp.random(), modulus).lift()
        while(gcd(a, modulus) != 1):
            a = mod(gp.random(), modulus).lift()
        yield a
        num += 1

class Alice:
    
    def __init__(self, modbits, seed):
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        # use gp.random() to get prng random number
    
        self.p = next_prime(2^modbits + gp.random())
        print("p : ", self.p)
        self.q = next_prime(2^modbits + gp.random())
        print("q : ", self.q)
        if self.p == self.q:
            print("Primes are same, please run again")
        self.n = self.p*self.q
        self.phi_n = (self.p-1)*(self.q-1)
        # Lifting is necessary here, in order for inverse_mod to work correctly
        self.e = mod(gp.random(), self.phi_n).lift()
        while gcd(self.e, self.phi_n)!=1:
            self.e = mod(gp.random(), self.phi_n).lift()
        self.d = inverse_mod(self.e, self.phi_n)
        
        # Alice generates m0 and m1 here.
        self.mlist = list(gen_random_mod(2, self.n))
        self.m0 = self.mlist[0]
        self.m1 = self.mlist[1]
        print("Alice's message m0 is : " + str(self.m0))
        print("Alice's message m1 is : " + str(self.m1))
        
        # Alice generates x0 and x1 here.
        self.xlist = list(gen_random_mod(2, self.n))
        
    def compute_kx(self, v):
        klist = []
        for i in range(2):
            klist.append(mod((v-self.xlist[i]), self.n).lift().powermod(self.d, self.n))
        #print(klist)
        mprimelist = []
        mprimelist.append(mod(self.m0 + klist[0], self.n).lift())
        mprimelist.append(mod(self.m1 + klist[1], self.n).lift())
        return mprimelist
        
        
class Bob:
    
    def __init__(self, xlist, n, e):
        self.x0 = xlist[0]
        self.x1 = xlist[1]
        self.xlist = xlist
        self.n = n
        self.e = e
        temp = [i-1 for i in list(gen_random_mod(1, 3))]
        self.b = temp[0]
        temp = list(gen_random_mod(1, self.n))
        self.k = temp[0]
        #print(self.k)
        self.v = mod(xlist[self.b], self.n)
        self.v = self.v + self.k.powermod(self.e, self.n)
        self.v = self.v.lift()
        
    def revelation(self, mprimelist):
        self.m = mprimelist[self.b] - self.k
        print("Bob's bit (0 or 1) b is : "+str(self.b))
        print("Bob could learn the value of m_b as : "+str(self.m))
        
        
class OT_Protocol:
    
    def __init__(self, modbits, seed):
        alice = Alice(modbits, seed)
        bob = Bob(alice.xlist, alice.n, alice.e)
        mprimelist = alice.compute_kx(bob.v)
        bob.revelation(mprimelist)
        if bob.m == alice.mlist[bob.b]:
            print("Protocol Finished Correctly!")
        else:
            print("Error in protocol resuts. Aborted.")
        
modbits = 12
ot = OT_Protocol(modbits, seed)
        

('p : ', 1565135459)
('q : ', 507035423)
Alice's message m0 is : 86256102
Alice's message m1 is : 1773138425
Bob's bit (0 or 1) b is : 1
Bob could learn the value of m_b as : 1773138425
Protocol Finished Correctly!
